<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE_TE_MobileNet_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import joblib
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
CONFIG = {
    'BASE_DIR'         : '/content/drive/MyDrive/Research/Crowd Emotion v4/',
    'DATASET_DIR'      : 'Dataset_AF/',
    'LOG_DIR'          : 'Logs/',
    'FOLDS'            : ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5'],
    'EMOTIONS'         : ['Approval', 'Disapproval', 'Neutral'],

    'IMG_SIZE'         : (160, 160),
    'DATASET_LEN'      : 14882,

    'SPEC_CONFIG'      : 'LOG_SPEC_W400_H200_HAMM_IMG160',

    'TEST_FOLD'        : 'Fold 1',
    'EPOCHS'           : 10,
    'LEARNING_RATE'    : 0.001,
}

In [3]:
path = os.path.join(CONFIG['BASE_DIR'], CONFIG['DATASET_DIR'])
X    = joblib.load(path + CONFIG['SPEC_CONFIG'] + '_X.joblib')
y    = joblib.load(path + CONFIG['SPEC_CONFIG'] + '_y.joblib')
f    = joblib.load(path + CONFIG['SPEC_CONFIG'] + '_f.joblib')

mask      = (f == CONFIG['FOLDS'].index(CONFIG['TEST_FOLD']))
train_X   = X[~mask, :]
test_X    = X[mask, :]
train_y   = y[~mask, :]
test_y    = y[mask, :]

In [4]:
class TransferLearning():
    def __init__(self, base_model, input_shape, learning_rate, batch_size, epochs):
        self.base_model = base_model
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        if base_model == 'mobile_net_v2':
            self.base_model = tf.keras.applications.MobileNetV2(
                input_shape     = self.input_shape,
                include_top     = False,
                weights         = 'imagenet'
            )
            self.base_model.trainable = False
            self.preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
            self.model = None
            self.callbacks = None

    def __init_model(self, num_classes):
        inputs = tf.keras.Input(shape=self.input_shape)
        x = self.preprocess(inputs)
        x = self.base_model(x, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = tf.keras.layers.Dense(num_classes)(x)
        self.model = tf.keras.Model(inputs, outputs)
        self.model.compile(
            optimizer      = tf.keras.optimizers.Adam(lr=self.learning_rate),
            loss           = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics        = ['accuracy']
        )

    def __init_callbacks(self):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor        = 'val_loss',
            patience       = 3,
            verbose        = 1
        )
        self.callbacks = [
                          early_stopping,
        ]

    def fit(self, train_X, train_y, val_X, val_y):
        num_classes = np.unique(train_y).shape[0]
        self.__init_model(num_classes)
        self.__init_callbacks()
        history = self.model.fit(
            x                 = train_X,
            y                 = train_y,
            batch_size        = self.batch_size,
            epochs            = self.epochs,
            verbose           = 1,
            validation_data   = (val_X, val_y),
            shuffle           = True,
            callbacks         = self.callbacks
        )
    
        return history 

    def evaluate(self, test_X, test_y):
        prob_model = tf.keras.Sequential([self.model, tf.keras.layers.Softmax()])
        y_pred_hot = prob_model.predict(test_X)
        y_pred = np.argmax(y_pred_hot, axis=1)
        return classification_report(test_y.ravel(), y_pred)

In [5]:
tl = TransferLearning(
    base_model         = 'mobile_net_v2',
    input_shape        = CONFIG['IMG_SIZE'] + (3,),
    learning_rate      = 0.0001,
    batch_size         = 32,
    epochs             = 100
)
history = tl.fit(train_X, train_y, test_X, test_y)

Epoch 1/100
375/375 [==============================] - 16s 30ms/step - loss: 0.9762 - accuracy: 0.5408 - val_loss: 0.8375 - val_accuracy: 0.5702
Epoch 2/100
375/375 [==============================] - 10s 26ms/step - loss: 0.6938 - accuracy: 0.7021 - val_loss: 0.8061 - val_accuracy: 0.5963
Epoch 3/100
375/375 [==============================] - 10s 26ms/step - loss: 0.5749 - accuracy: 0.7726 - val_loss: 0.8189 - val_accuracy: 0.5935
Epoch 4/100
375/375 [==============================] - 10s 26ms/step - loss: 0.5046 - accuracy: 0.8060 - val_loss: 0.7986 - val_accuracy: 0.6060
Epoch 5/100
375/375 [==============================] - 10s 26ms/step - loss: 0.4608 - accuracy: 0.8299 - val_loss: 0.8183 - val_accuracy: 0.6060
Epoch 6/100
375/375 [==============================] - 10s 26ms/step - loss: 0.4189 - accuracy: 0.8421 - val_loss: 0.8495 - val_accuracy: 0.6032
Epoch 7/100
375/375 [==============================] - 10s 26ms/step - loss: 0.3915 - accuracy: 0.8573 - val_loss: 0.8859 - val_ac

In [10]:
result = tl.evaluate(test_X, test_y)

In [11]:
print(result)

              precision    recall  f1-score   support

           0       0.56      0.78      0.65      1335
           1       0.57      0.52      0.54       390
           2       0.74      0.42      0.54      1158

    accuracy                           0.60      2883
   macro avg       0.62      0.57      0.58      2883
weighted avg       0.63      0.60      0.59      2883

